In [ ]:
import pennylane as qml
from pennylane import numpy as np
import time

def local_hadamard_test(weights, problem, l=None, lp=None, j=None, part=None):

    ancilla_idx = problem.get_n_qubits()

    # First Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # For estimating the imaginary part of the coefficient "mu", we must add a "-i"
    # phase gate.
    if part == "Im" or part == "im":
        qml.PhaseShift(-np.pi / 2, wires=ancilla_idx)

    # Variational circuit generating a guess for the solution vector |x>
    problem.variational_block(weights)

    # Controlled application of the unitary component A_l of the problem matrix A.
    problem.CA(ancilla_idx, l)

    # Adjoint of the unitary U_b associated to the problem vector |b>.
    # In this specific example Adjoint(U_b) = U_b.
    problem.U_b()

    # Controlled Z operator at position j. If j = -1, apply the identity.
    if j != -1:
        qml.CZ(wires=[ancilla_idx, j])

    # Unitary U_b associated to the problem vector |b>.
    problem.U_b()

    # Controlled application of Adjoint(A_lp).
    # In this specific example Adjoint(A_lp) = A_lp.
    problem.CA(ancilla_idx, lp)

    # Second Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=ancilla_idx))

# Computes the mu coefficients
def mu(weights, local_hadamard_test, problem, l=None, lp=None, j=None):
    """Generates the coefficients to compute the "local" cost function C_L."""

    mu_real = local_hadamard_test(weights, problem, l=l, lp=lp, j=j, part="Re")
    mu_imag = local_hadamard_test(weights, problem, l=l, lp=lp, j=j, part="Im")

    return mu_real + 1.0j * mu_imag

def psi_norm(weights, c, local_hadamard_test, problem):
    """Returns the normalization constant <psi|psi>, where |psi> = A |x>."""
    norm = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            norm = norm + c[l] * np.conj(c[lp]) * mu(weights, local_hadamard_test, problem, l, lp, -1)

    return abs(norm)

def cost_loc(problem, weights, local_hadamard_test):
    """Local version of the cost function. Tends to zero when A|x> is proportional to |b>."""

    c, _ = problem.get_coeffs()
    n_qubits = problem.get_n_qubits()
    
    mu_sum = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            for j in range(0, n_qubits):
                mu_sum = mu_sum + c[l] * np.conj(c[lp]) * mu(weights, local_hadamard_test, problem, l, lp, j)

    mu_sum = abs(mu_sum)

    # Cost function C_L
    res = 0.5 - 0.5 * mu_sum / (n_qubits * psi_norm(weights, c, local_hadamard_test, problem))

    return res

# convert matrix A encoded as a string (eg. "IZZ") into qml code
def A_to_code (idx, ancilla_idx, terms, offset=0):

    if idx < 0:
        raise ValueError("Index of linear combination must be >= 0.")
    
    target_pauli = list(terms[idx])
    
    order_idx = offset

    for i in range(len(target_pauli)):
        if target_pauli[i] == 'I':
            order_idx += 1
            None
        if target_pauli[i] == 'X':
            qml.CNOT(wires = (ancilla_idx, order_idx))
            order_idx += 1
        if target_pauli[i] == 'Y':
            qml.CY(wires = (ancilla_idx, order_idx))
            order_idx += 1
        if target_pauli[i] == 'Z':
            qml.CZ(wires = (ancilla_idx, order_idx))
            order_idx += 1

import functools as ft

pauli_dict = {"I": qml.Identity.compute_matrix(), "X": qml.PauliX.compute_matrix(), "Y": qml.PauliY.compute_matrix(), "Z": qml.PauliZ.compute_matrix()}

def A_to_num (n_qubits: int, coefs: np.tensor, terms):
    """
    Given an array of coeffs c and an array of A_l formatted as a list of strings, return A
    @params
    coefs (eg. [1, 0.2, 0.2])
    terms (eg. ["III", "XZI", "XII"])

    returns an np.array
    """    
    if len(coefs) != len(terms):
        raise ValueError("Number of coefficients does not match number of terms.")
    
    if n_qubits <= 0:
        raise ValueError("Number of qubits is not a number greater than 0.")
    
    terms_len = len(terms)
    for i in range(terms_len):
        if len(terms[i]) != n_qubits:
            raise ValueError("Number of terms in each Pauli gate combination must be the same as number of qubits.")
        

    dim = 2**n_qubits
    mat = np.zeros((dim, dim), dtype=np.complex64)

    for (c, pauli) in zip(coefs, terms):
        pauli = [pauli_dict[key] for key in pauli]
        if pauli == ["I"]:
            mat += c * ft.reduce(np.kron, pauli)
        else:
            mat += c * ft.reduce(np.kron, pauli)
        
    return mat


# these classes encode the linear system we are trying to solve
from abc import ABC, abstractmethod
class Problem(ABC):
    def __init__(self, n_qubits, c, A_terms) -> None:
        super().__init__()
        self.n_qubits = n_qubits
        self.A_num = A_to_num(n_qubits, c, A_terms)
        self.A_terms = A_terms

        # normalize c
        self.c = np.array(c) / np.linalg.norm(self.A_num, ord=2)

        # Total number of qubits; here we add an ancillary qubit
        self.tot_qubits = self.n_qubits + 1
        # Index of ancillary qubit (Python lists are 0-indexed)
        self.ancilla_idx = self.n_qubits

    @abstractmethod
    def get_coeffs():
        """gets c, A_l"""
        pass
    
    @abstractmethod
    def get_n_qubits():
        """gets number of qubits of your problem"""
        pass

    @abstractmethod
    def U_b():
        """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
        pass

    @abstractmethod
    def CA(idx):
        pass

    @abstractmethod
    def variational_block(weights):
        pass

class ToyProblem(Problem):
    def __init__(self, n_qubits):
        c = [1, 0.25]
        A_terms = ["III", "IIZ"]

        super().__init__(n_qubits, c, A_terms)

        self.param_shape = n_qubits

    def get_coeffs(self):
        return self.c, self.A_terms
    
    def get_n_qubits(self):
        return self.n_qubits
        

    def U_b(self):
        """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
        [qml.Hadamard(wires=i) for i in [0,1]]
        
    def CA(self, ancilla_idx, idx, offset=0):
        A_to_code(idx, ancilla_idx=ancilla_idx, terms=self.A_terms, offset=offset)

    def variational_block(self, weights, offset=0):
        [qml.RY(phi=weights[i], wires=i+offset) for i in range(self.n_qubits)]

#############
n_qubits = 3
dev_mu = qml.device("default.qubit", wires=n_qubits+1)

from pennylane.transforms import mitigate_with_zne

scale_factors = [1, 2, 3]

local_hadamard_test = qml.QNode(local_hadamard_test, dev_mu, interface="autograd")
local_hadamard_test = mitigate_with_zne(local_hadamard_test, scale_factors, qml.transforms.fold_global, qml.transforms.richardson_extrapolate) # comment this line out to remove the error mitigation

problem = ToyProblem(3)
opt = qml.GradientDescentOptimizer(0.1)
w = np.random.randn(problem.param_shape, requires_grad=True)

start = time.time()

for it in range(10):
    # w, cost = opt.step_and_cost(cost_agg, w)
    w, cost = opt.step_and_cost(lambda w: cost_loc(problem, w, local_hadamard_test), w)

    print("Step {:3d}       Cost_L = {:9.7f}".format(it, cost), flush=True)

    it += 1

print(f"Training time: {time.time() - start}s")